# Part 2 of the end-to-end example

In the last notebook, we explored the data and did some feature engineering. In this notebook, we'll apply the basic processing steps from the previous one, then train and fine-tune a model.

Ultimately this repeated code would probably be better as a separate script, but I'm being lazy and copy pasting.

In [ ]:
# Load the data

from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    dir_path = Path("../datasets")
    tarball_path = dir_path / "housing.tgz"
    if not tarball_path.is_file():
        dir_path.mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path=dir_path)
    return pd.read_csv(dir_path / "housing" / "housing.csv")

housing = load_housing_data()

In [ ]:
# Stratified sampling

from sklearn.model_selection import train_test_split
import numpy as np

# Add the income category and do our stratified sampling
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)

# We don't actually want the income_cat column sticking around, so let's drop it
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# Do our feature engineering
housing["rooms_per_house"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_ratio"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["people_per_house"] = housing["population"] / housing["households"]

## Prepare the Data for Machine Learning Algorithms

We don't want the target (the median house value) to be in the predictors, so we'll split the training set into predictor and target (often called $x$ and $y$).

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

## Data Cleaning

### Missing features
Most ML algorithms can't deal with missing features, so we need to do so during the wrangling step.

In the book 3 options are listed to handle the NaN values:

```python
housing.dropna(subset=["total_bedrooms"], inplace=True)    # option 1

housing.drop("total_bedrooms", axis=1)       # option 2

median = housing["total_bedrooms"].median()  # option 3
housing["total_bedrooms"].fillna(median, inplace=True)
```

For each option, we'll create a copy of `housing` and work on that copy to avoid breaking `housing`. We'll also show the output of each option, but filtering on the rows that originally contained a NaN value.

**❓ Discussions for class:**
- What is each option doing?
- What are the pros and cons of each option?
- Which one should we choose?

In [ ]:
# First, let's look at the missing values in the total_bedrooms column
null_rows_idx = housing.isnull().any(axis=1)
print(f"Missing {len(housing[null_rows_idx])} of {len(housing)} rows")
housing.loc[null_rows_idx].head()

Replacing the NaN values with the median is surprisingly common, as it preserves the most data. Scikit-learn provides a `SimpleImputer` class to do this automatically. There's also `KNNImputer` which uses the $k$ nearest neighbors to fill in the missing values, or `IterativeImputer` which repeatedly trains a regression model to predict the missing values.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

It doesn't work on text data, so we need to filter the dataframe to just include numeric types. After that, we can use the imputer as a **transformer** to fill in the missing values.

In [ ]:
housing_num = housing.select_dtypes(include=[np.number])
imputer.fit(housing_num)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)
housing_tr.loc[null_rows_idx].head()

In [ ]:
from sklearn import set_config

set_config(transform_output="pandas")  # scikit-learn >= 1.2

### Outliers

As we saw on the various scatter plots, there's some outliers in the data. One way to detect them is to use an [isolation forest](https://doi.org/10.1109/ICDM.2008.17), which recursively splits the data into subsets until each contains only one sample. Outliers are samples that require fewer splits to isolate. The `IsolationForest` class in scikit-learn implements this algorithm and can be used to train a model to predict an anomaly score for each sample, with -1 being an outlier and 1 being an inlier.

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)
outlier_pred[outlier_pred == -1].size / outlier_pred.size

If you wanted to drop outliers, you would run the following code:

In [ ]:
#housing = housing.iloc[outlier_pred == 1]
#housing_labels = housing_labels.iloc[outlier_pred == 1]

### Handling Text and Categorical Attributes

Most of the math in ML algorithms is based on numbers, so we need to convert text and categorical attributes to numbers. This is called **encoding**.

**❓ Discussions for class:**
- Which columns of our data are categorical?
- What methods could we use to convert them to numbers?

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(8)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

plt.hist(housing_cat_encoded)
plt.xticks(range(5), ordinal_encoder.categories_[0])

**❓ Discussions for class:**
- What does the `OrdinalEncoder` assume?
- Does it make sense in this context?

An alternative method is a **One-hot encoder**, which creates a new binary attribute for each category. This can get really big if there are a lot of categories, but it's fine for this dataset.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse_output=False) # pandas doesn't like spare matrices
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.head()

### Feature Scaling

The last piece of data cleaning that we want to look at is feature scaling. Most ML algorithms don't like hugely different scales, so we want to scale the features to be similar. Two popular methods are **min-max scaling** and **standardization**:

- Min-max scaling (also called normalization) shifts and rescales the data so that it ends up ranging from 0 to 1.
- Standardization subtracts the mean value and divides by the standard deviation so that the resulting distribution has unit variance (aka the **standard normal distribution**).

**🧮 Math break!**

A general Gaussian distribution is given by:

$$ f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2} \left(\frac{x - \mu}{\sigma}\right)^2} $$

where $\mu$ is the mean and $\sigma$ is the standard deviation. The standard normal distribution is a special case where $\mu = 0$ and $\sigma = 1$, reducing the equation to:

$$ f(x) = \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2} x^2} $$

In [ ]:
def gaussian(x, mu, sigma):
    return np.exp(-((x - mu) ** 2) / (2 * sigma ** 2)) / np.sqrt(2 * np.pi * sigma ** 2)

x = np.linspace(-5, 7, 100)
plt.plot(x, gaussian(x, 0, 1), label="$\mu=0$, $\sigma=1$")
plt.plot(x, gaussian(x, 1, 2), label="$\mu=1$, $\sigma=2$")
plt.legend()
plt.xlabel("x")
plt.ylabel("f(x)")


The normal distribution is important because of the **central limit theorem**, which says that the **sum** of a large number of **independent and identically distributed** (i.i.d) random variables is approximately normally distributed. For example, consider the simplest possible random variable: the Bernoulli distribution, which has two possible outcomes (0 or 1) with probabilities $p$ and $1-p$ respectively.

In the previous notebook, we saw that the average of $n$ Bernoulli random variables is described by the binomial distribution:

$$ f(k;n,p) = \Pr(k;n,p) = \Pr(X = k) = \binom{n}{k} p^k (1-p)^{n-k} $$

where $k$ is the number of successes, $n$ is the number of trials, and $p$ is the probability of success. The **mean** or **expected value** of the binomial distribution is $np$ and the **variance** is $np(1-p)$.

The central limit theorem says that if we take the average of $n$ Bernoulli random variables, the distribution of the average will be approximately normal, with the same mean and standard deviation.

> Important! This works for the binomial distribution because it is already the **sum** of i.i.d. random variables. The CLT does **not** say that as $n$ gets large, the distribution of **any** random variable will be approximately normal.

In [ ]:
from scipy.stats import binom, norm

p = 0.3 # let's say the average student has a 30% chance of falling asleep in class
n = 6 # there's only 6 students in the class

x = np.arange(-1, n + 1)
plt.bar(x, binom.pmf(x, n, p), label="Binomial pmf")
plt.plot(x, norm.pdf(x, n * p, np.sqrt(n * p * (1 - p))), color="red", label="Normal approximation")
plt.xlabel("Number of students asleep")
plt.ylabel("Probability")


Back to the task at hand... here's how you would normalize the data using scikit-learn's `MinMaxScaler`, or standardize it using `StandardScaler`. First, let's recall the distributions of the features:

In [ ]:
housing.hist(bins=50, figsize=(12, 8))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num)
housing_num_min_max_scaled.hist(bins=50, figsize=(12, 8))

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(housing_num)
housing_num_std_scaled.hist(bins=50, figsize=(12, 8))

**❓ Discussions for class:**
- What are the bounds of each method?
- Which method is more affected by outliers?
- How would you decide which method to use?

### Other transformations
Sometimes you might want to do things like log-transform the data to deal with long tails (skewness), or even convert a numerical attribute to a categorical one to deal with a multi-modal distribution (like `latitude` above). The book goes into details on a few more examples, but the main takeaway is that there is no rule that says you need to feed your data as-is into an ML algorithm, and data transformations might make sense.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted

class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean=True):  # no *args or **kwargs!
        self.with_mean = with_mean

    def fit(self, X, y=None):  # y is required even though we don't use it
        X = check_array(X)  # checks that X is an array with finite float values
        self.mean_ = X.mean(axis=0)
        self.scale_ = X.std(axis=0)
        self.n_features_in_ = X.shape[1]  # every estimator stores this in fit()
        return self  # always return self!

    def transform(self, X):
        check_is_fitted(self)  # looks for learned attributes (with trailing _)
        X = check_array(X)
        assert self.n_features_in_ == X.shape[1]
        if self.with_mean:
            X = X - self.mean_
        return X / self.scale_

### Transformation Pipelines

Data transformations are so common that most frameworks have a way of handling them efficiently. We'll set up a pipeline of functional transformations that will be applied to the data in sequence. A key thing to note is that the pipeline is fit to the **training data** - you should never re-normalize based on test data, for example.

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

# let's go with the median imputer to fill in the missing total_bedrooms value
# Use the standard scaler for normalization
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

In [ ]:
# Note: the book and original version of this notebook include some fancier methods of
# composing pipelines. I've removed them as it's all just syntax.
from sklearn.compose import ColumnTransformer

num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False))

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [ ]:
# Fit the transform to the housing data, which is just the training set
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared.head()

## Select and Train a Model
Finally we're at the actual modelling stage! We'll start with linear regression, then try a decision tree. No use getting fancy unless we know the simple things don't work.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(housing, housing_labels)

Let's use the `predict` method on the **training** set to see how well our model does. This is kind of a sanity check - if our performance is bad on the training set, it certainly won't be good on the test set.

In [ ]:
# The book just inspects a few values, but I prefer visualizing
def plot_predictions(actual, pred):
    plt.scatter(actual, pred, alpha=0.1)
    plt.plot([0, 500000], [0, 500000], color="red")
    plt.xlabel("Actual Median House Value ($)")
    plt.ylabel("Predicted Median House Value ($)")

housing_predictions = lin_reg.predict(housing)
plot_predictions(housing_labels, housing_predictions)
plt.title("Linear Regression Prediction")

In [ ]:
# calculate the RMSE
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_labels, housing_predictions,
                              squared=False)
lin_rmse

Okay, that wasn't great. Let's move on to a decision tree.

In [ ]:
# Scikit-learn makes it pretty easy to swap out a model in the same pipeline
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(housing, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing)

plot_predictions(housing_labels, housing_predictions)
plt.title("Decision Tree Prediction")

tree_rmse = mean_squared_error(housing_labels, housing_predictions,
                              squared=False)
tree_rmse # wtf?

**❓ Discussions for class:**
- What is happening here?
- What is under and over fitting?

It's tempting to look at the performance on the test set at this point, but this can actually influence your choice of model! Best to keep the test set locked away until you're satisfied with the model. Rest assured that an RMSE of 0.0 is not a good sign.

### Better Evaluation Using Cross-Validation
Rather than just a train/test split, we can add a third set called **validation**. This is a subset of the training set that is used to evaluate the model during training rather than to train parameters.

**Cross-validation** is a technique where you split the training set $k$ times into a training/validation set, then train the model $k$ times. For example, if $k = 10$ you split the training set into 10 **non-overlapping** folds, pick one of them as the validation set and train on the other 9, then evaluate on the first. Repeat 10 times and you get 10 different models. The final model is the average of the 10 models.

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict

cv_predictions = cross_val_predict(tree_reg, housing, housing_labels, cv=10)

In [ ]:
plot_predictions(housing_labels, cv_predictions)
plt.title("Decision Tree 10-fold Cross-Validated Prediction")
cv_mse = mean_squared_error(housing_labels, cv_predictions, squared=False)
cv_mse


Let's try one more model: random forest. This is an **ensemble** of decision trees, where each one is trained on a random subset of the training data and a random subset of the **features**. The final prediction is the average of the predictions of all the trees.

**Warning:** the following cell may take a few minutes to run:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing,
                           RandomForestRegressor(random_state=42))
forest_reg.fit(housing, housing_labels)

In [ ]:
forest_predictions = forest_reg.predict(housing)
plot_predictions(housing_labels, forest_predictions)
plt.title("Random Forest Prediction")
forest_rmse = mean_squared_error(housing_labels, forest_predictions, squared=False)
forest_rmse

Hmm... not quite 0, but looks a bit too good to be true. Let's try measuring the cross-validation score (this is going to take a while, as it's training 10 random forest models):

In [ ]:
cv_forest_score = -cross_val_score(forest_reg, housing, housing_labels, 
                                   scoring="neg_root_mean_squared_error", cv=10)
cv_forest_score

The training error is much lower than the validation error, which usually means that the model has overfit the training set. Another possible explanation may be that there's a mismatch between the training data and the validation data, but it's not the case here, since both came from the same dataset that we shuffled and split in two parts.

**❓ Discussions for class:**
- What are some ways to reduce overfitting?
- What other models could we try for this task?
- Is there anything in our preprocessing pipeline that we could change?

Ultimately: garbage in, garbage out. Data quality is paramount!

## Fine-Tune Your Model
Let's assume that we tried a bunch of things and are forging ahead with the random forest model. We *could* tweak our hyperparameters (parameters about the model rather than model parameters) manually, but we can also be systematic about it.

### Grid Search
One option is to manually define the combinations of hyperparameters that we want to try, then train and evaluate the model on each combination. This is called **grid search**. It's straightforward, but expensive.

**Warning:** the following cell may take a few minutes to run:

In [ ]:
from sklearn.model_selection import GridSearchCV

full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42)),
])
# Note: the original version of this notebook and the book reference the
# geo clustering parameters that we skipped.
param_grid = [
    {'random_forest__max_features': [4, 6, 8]},
]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring='neg_root_mean_squared_error')
grid_search.fit(housing, housing_labels)

You can get the full list of hyperparameters available for tuning by looking at `full_pipeline.get_params().keys()`:

In [ ]:
# extra code – shows part of the output of get_params().keys()
print(str(full_pipeline.get_params().keys())[:1000] + "...")

The best hyperparameter combination found:

In [ ]:
grid_search.best_params_

Let's look at the score of each hyperparameter combination tested during the grid search: (not really a combination now that I've reduced it to one hyperparameter)

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

In [ ]:
# Take a look at the final model behaviour
grid_search_predictions = grid_search.best_estimator_.predict(housing)
plot_predictions(housing_labels, grid_search_predictions)
plt.title("Grid Search Random Forest Prediction")
grid_search_rmse = mean_squared_error(housing_labels, grid_search_predictions, squared=False)
grid_search_rmse # probably still overfitting

In [ ]:
final_model = grid_search.best_estimator_  # includes preprocessing
plt.bar(final_model["preprocessing"].get_feature_names_out(), final_model["random_forest"].feature_importances_)
plt.xticks(rotation=90)
plt.xlabel("Feature")
plt.ylabel("Importance")

### Final model evaluation
In the end, the grid search didn't help all that much, probably because it wasn't a true grid. The book also goes over some **randomized search** methods that are more efficient than grid search, but we'll skip that for now. There's also `HalvingRandomSearchCV` and `HalvingGridSearchCV` which do some more intelligent searching, but you could spend forever on this.

**❓ Discussions for class:**
- Which features were the most important?
- Could we drop or combine any features to simplify the model?
- Are there any other models we could try?

## Evaluate Your System on the Test Set
We'll assume we're happy with the final model and want to deploy it. The last step is to evaluate it on the test set to get an estimate of the **generalization error**.

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

grid_search_predictions = final_model.predict(X_test)

final_rmse = mean_squared_error(y_test, grid_search_predictions, squared=False)
plot_predictions(y_test, grid_search_predictions)
plt.title("Test Set Prediction")
final_rmse

This is a higher number than in the book, but we skipped a few steps. However, we have to be careful with interpreting a single number - much better to look at the **confidence interval**.

**🧮 Math break!**
A [confidence interval](https://en.wikipedia.org/wiki/Confidence_interval) defines a range of values that tend towards a given confidence of containing the value. Assuming that the values are normally distributed (thanks CLT), we can [compute the confidence interval](https://en.wikipedia.org/wiki/Normal_distribution#Confidence_intervals) as:

$$\left[ \hat\mu - t_{n-1,1-\alpha/2} \frac{1}{\sqrt{n}}s,
                      \hat\mu + t_{n-1,1-\alpha/2} \frac{1}{\sqrt{n}}s \right]$$

where $\hat\mu$ is the sample mean, $t_{n-1,\alpha/2}$ is the $t$-score for confidence $1 - \alpha$, $s$ is the sample standard deviation, and $n$ is the number of samples. $n-1$ is referred to as the **degrees of freedom**, and represents the logical maximum number of independent values that can be chosen freely.

Back in the day we needed to use $t$-tables and interpolation, but now it's pretty easy to compute:

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (grid_search_predictions - y_test) ** 2
n = len(squared_errors)
np.sqrt(stats.t.interval(confidence, n - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

We could also do it manually like this:

In [ ]:
# extra code – shows how to compute a confidence interval for the RMSE
mean = squared_errors.mean()
tscore = stats.t.ppf((1 - confidence) / 2, df=n - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(n)
np.sqrt(mean + tmargin), np.sqrt(mean - tmargin)

Alternatively, we could use a z-score rather than a t-score if the dataset is big enough. This assumes that the estimate of the standard deviation is representative of the population, which is not always the case.

In [ ]:
# extra code – computes a confidence interval again using a z-score
zscore = stats.norm.ppf((1 - confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(n)
np.sqrt(mean + zmargin), np.sqrt(mean - zmargin)

## Model persistence using joblib
After all this, you want a useful model that you can deploy in production. The last step is to save all those parameters you just trained so that you can load them later. Pickle could be used for this, but `joblib` is more efficient.

In [ ]:
import joblib

joblib.dump(final_model, "my_california_housing_model.pkl")

Now you can deploy this model to production. For example, the following code could be a script that would run in production:

In [ ]:
import joblib

# extra code – excluded for conciseness
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel

def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

#class ClusterSimilarity(BaseEstimator, TransformerMixin):
#    [...]

final_model_reloaded = joblib.load("my_california_housing_model.pkl")

new_data = housing.iloc[:5]  # pretend these are new districts
predictions = final_model_reloaded.predict(new_data)
predictions

And that's the basic process! The rest of this course will dig in to the details of each step and look at a number of different ways of representing data, different models, and some of the math hiding behind it all.